# Data Smart Sans Excel

[Table of Contents](Data%20Smart%20Sans%20Excel.ipynb)

If you have not downloaded the Data Smart files then run the first code block of the main notebook and it will download the files from the web. 

## Chapter 3 - Optimization Modeling

In [4]:
import os
import pandas as pd
import numpy as np
excel_file = os.path.join(os.getcwd(), "data_smart_files", "ch04", "OrangeJuiceBlending.xlsx")
specs_df = pd.read_excel(excel_file, 'Specs', parse_cols = "A:I", index_col=0)

* Objective: Minimize procurement costs.
* Decisions: Amount of each juice to buy each month
* Constraints:
    * Demand (600k in Jan/Feb, 700k in Mar)
    * Supply
    * Florida Valencia requirement: Buy 40% of juice from FL.
    * Flavor: 
        * 11.5 < BAR < 12.5
        * 0.75 < Acid < 1
        * Astrigency <= 4
    * Color: 4.5 < Color < 5.5

In [5]:
print (specs_df)

                    Region  Qty Available (1,000 Gallons)  Brix / Acid Ratio  \
Varietal                                                                       
Hamlin              Brazil                            672               10.5   
Mosambi              India                            400                6.5   
Valencia           Florida                           1200               12.0   
Hamlin          California                            168               11.0   
Gardner            Arizona                             84               12.0   
Sunstar              Texas                            210               10.0   
Jincheng             China                            588                9.0   
Berna                Spain                            168               15.0   
Verna               Mexico                            300                8.0   
Biondo Commune       Egypt                            210               13.0   
Belladonna           Italy              

In [ ]:
total_required = [600, 600, 700]
valencia_required = [0.4*x for x in total_required]
    
quality_cons_min = [11.5, 0.0075, 0, 4.5]
quality_cons_max = [12.5, 0.01, 4, 5.5]

In [102]:
def opti(x, baseline_data):
    
    opti_model = baseline_data
    
    opti_model['Jan Orders'] = x[:11]
    opti_model['Feb Orders'] = x[11:22]
    opti_model['Mar Orders'] = x[22:33] 

    opti_model['Jan Price'] = opti_model['Jan Orders'] * opti_model['Price (per 1K Gallons)']
    opti_model['Feb Price'] = opti_model['Feb Orders'] * opti_model['Price (per 1K Gallons)']
    opti_model['Mar Price'] = opti_model['Mar Orders'] * opti_model['Price (per 1K Gallons)']
    
    opti_model['Jan Shipping'] = opti_model['Jan Orders'] * opti_model['Shipping']
    opti_model['Feb Shipping'] = opti_model['Feb Orders'] * opti_model['Shipping']
    opti_model['Mar Shipping'] = opti_model['Mar Orders'] * opti_model['Shipping']
   
    o_i = list(opti_model.index)    
    o_i.append('Total')    
    opti_model = opti_model.append(opti_model.sum(numeric_only=True), ignore_index=True)
    opti_model['I'] = o_i
    opti_model.set_index('I', drop=True, inplace=True)

    total_cost = opti_model.at['Total','Jan Price'] + opti_model.at['Total','Feb Price'] + opti_model.at['Total','Mar Price'] + opti_model.at['Total', 'Jan Shipping'] + opti_model.at['Total','Feb Shipping'] + opti_model.at['Total','Mar Shipping']

    return total_cost

In [97]:
#def cons(x, baseline):
#    opti_model = baseline_data
    
#    opti_model['Jan Orders'] = x[:11]
#    opti_model['Feb Orders'] = x[11:22]
#    opti_model['Mar Orders'] = x[22:33] 
    
    #opti_model['Jan Price'] = opti_model['Jan Orders'] * opti_model['Price (per 1K Gallons)']
    #opti_model['Feb Price'] = opti_model['Feb Orders'] * opti_model['Price (per 1K Gallons)']
    #opti_model['Mar Price'] = opti_model['Mar Orders'] * opti_model['Price (per 1K Gallons)']
    
    #opti_model['Jan Shipping'] = opti_model['Jan Orders'] * opti_model['Shipping']
    #opti_model['Feb Shipping'] = opti_model['Feb Orders'] * opti_model['Shipping']
    #opti_model['Mar Shipping'] = opti_model['Mar Orders'] * opti_model['Shipping']
   
    #o_i = list(opti_model.index)    
    #o_i.append('Total')    
    #opti_model = opti_model.append(opti_model.sum(numeric_only=True), ignore_index=True)
    #opti_model['I'] = o_i
    #opti_model.set_index('I', drop=True, inplace=True)
    
    #opti_model.at['Total','Jan Orders'] + opti_model.at['Total','Feb Orders'] + opti_model.at['Total','Mar Orders']
    
    
#    total_required = [600, 600, 700]
#    valencia_required = [0.4*x for x in total_required]
    
#    quality_cons_min = [11.5, 0.0075, 0, 4.5]
#    quality_cons_max = [12.5, 0.01, 4, 5.5]
#    
    
    
    #quality_df = pd.DataFrame(np.zeros(4, 5), cols=['Jan','Feb','Mar','Min','Max'], 
    #                         index=['BAR', 'Acid', 'Astrigency', 'Color'])
    


In [105]:
from scipy.optimize import minimize

def orders_check(o, req):
    return o.sum() - req

def qual_cons_compare(orders,baseline, measure, mvm, constraint, total):
    return mvm*(constraint - ((orders * baseline[measure]).sum() / total))

cons = [{'type': 'eq', 'fun': lambda x: orders_check(x[:11],total_required[0])},
        {'type': 'eq', 'fun': lambda x: orders_check(x[11:22],total_required[1])},
        {'type': 'eq', 'fun': lambda x: orders_check(x[22:33],total_required[2])},
        {'type': 'eq', 'fun': lambda x: orders_check(x[3],valencia_required[0])},
        {'type': 'eq', 'fun': lambda x: orders_check(x[14],valencia_required[1])},
        {'type': 'eq', 'fun': lambda x: orders_check(x[25],valencia_required[2])},                                            
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[:11],specs_df,'Brix / Acid Ratio',1,quality_cons_max[0],total_required[0])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[:11],specs_df,'Acid (%)',1,quality_cons_max[1],total_required[0])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[:11],specs_df,'Astringency (1-10 Scale)',1,quality_cons_max[2],total_required[0])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[:11],specs_df,'Color (1-10 Scale)',1,quality_cons_max[3],total_required[0])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[11:22],specs_df,'Brix / Acid Ratio',1,quality_cons_max[0],total_required[1])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[11:22],specs_df,'Acid (%)',1,quality_cons_max[1],total_required[1])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[11:22],specs_df,'Astringency (1-10 Scale)',1,quality_cons_max[2],total_required[1])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[11:22],specs_df,'Color (1-10 Scale)',1,quality_cons_max[3],total_required[1])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[22:33],specs_df,'Brix / Acid Ratio',1,quality_cons_max[0],total_required[2])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[22:33],specs_df,'Acid (%)',1,quality_cons_max[1],total_required[2])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[22:33],specs_df,'Astringency (1-10 Scale)',1,quality_cons_max[2],total_required[2])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[22:33],specs_df,'Color (1-10 Scale)',1,quality_cons_max[3],total_required[2])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[:11],specs_df,'Brix / Acid Ratio',-1,quality_cons_min[0],total_required[0])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[:11],specs_df,'Acid (%)',-1,quality_cons_min[1],total_required[0])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[:11],specs_df,'Astringency (1-10 Scale)',-1,quality_cons_min[2],total_required[0])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[:11],specs_df,'Color (1-10 Scale)',-1,quality_cons_min[3],total_required[0])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[11:22],specs_df,'Brix / Acid Ratio',-1,quality_cons_min[0],total_required[1])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[11:22],specs_df,'Acid (%)',-1,quality_cons_min[1],total_required[1])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[11:22],specs_df,'Astringency (1-10 Scale)',-1,quality_cons_min[2],total_required[1])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[11:22],specs_df,'Color (1-10 Scale)',-1,quality_cons_min[3],total_required[1])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[22:33],specs_df,'Brix / Acid Ratio',-1,quality_cons_min[0],total_required[2])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[22:33],specs_df,'Acid (%)',-1,quality_cons_min[1],total_required[2])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[22:33],specs_df,'Astringency (1-10 Scale)',-1,quality_cons_min[2],total_required[2])},
        {'type': 'ineq', 'fun': lambda x: qual_cons_compare(x[22:33],specs_df,'Color (1-10 Scale)',-1,quality_cons_min[3],total_required[2])},
        {'type': 'ineq', 'fun': lambda x: x[:11] - specs_df['Qty Available (1,000 Gallons)'].values}]



#x0 = np.random.random_integers(0,100,(33)) 
x0 = np.zeros(33)

x0[:11] = total_required[0] / 11
x0[11:22] = total_required[1] / 11
x0[22:33] = total_required[2] / 11

print (x0)

res = minimize(opti, x0, method='SLSQP', args=specs_df, constraints=cons)

print (res)

[ 54.54545455  54.54545455  54.54545455  54.54545455  54.54545455
  54.54545455  54.54545455  54.54545455  54.54545455  54.54545455
  54.54545455  54.54545455  54.54545455  54.54545455  54.54545455
  54.54545455  54.54545455  54.54545455  54.54545455  54.54545455
  54.54545455  54.54545455  63.63636364  63.63636364  63.63636364
  63.63636364  63.63636364  63.63636364  63.63636364  63.63636364
  63.63636364  63.63636364  63.63636364]
     fun: -2097436715.0218334
     jac: array([ 576.,  416.,  736.,  672.,  672.,  640.,  512.,  704.,  352.,
        608.,  608.,  608.,  480.,  768.,  704.,  704.,  672.,  544.,
        672.,  416.,  576.,  608.,  608.,  464.,  768.,  704.,  640.,
        672.,  560.,  672.,  368.,  576.,  608.,    0.])
 message: 'Positive directional derivative for linesearch'
    nfev: 2559
     nit: 77
    njev: 73
  status: 8
 success: False
       x: array([  5.53451082e+01,   5.41734251e+01,   5.46003764e+01,
         5.41173041e+01,   5.39381910e+01,   5.51342418e+